In [18]:
# Allows us to import packages that exist one level up in the file system
# See https://stackoverflow.com/questions/34478398
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path = [module_path] + sys.path

In [2]:
from tagnews.utils import load_data as ld
import numpy as np
import pandas as pd

In [3]:
df = ld.load_data()
have_locs = df[df.locations.apply(len) > 0]

In [4]:
set_of_locs = have_locs.locations.iloc[0]
set_of_locs

[{'cleaned span': (60, 74),
  'cleaned text': 'Southwest Side',
  'end': 83,
  'start': 69,
  'text': 'Southwest Side'},
 {'cleaned span': (402, 433),
  'cleaned text': '6600 block of South Springfield',
  'end': 442,
  'start': 411,
  'text': '6600 block of South Springfield'},
 {'cleaned span': (466, 497),
  'cleaned text': '6300 block of South Springfield',
  'end': 506,
  'start': 475,
  'text': '6300 block of South Springfield'}]

In [16]:
import re
from geopy import geocoders
g = geocoders.GoogleV3(api_key = 'MY-KEY')

def clean(address):
    address = str(address)
    text = re.sub(r'(?:\@|https?\://)\S+', "", address)
    return text

def get_lat_longs(set_of_locs):
    for loc in set_of_locs:
        address = clean(loc['cleaned text']) # + ', Chicago'
        try: 
            location = g.geocode(address, timeout=10)
            loc['latitude'], loc['longitude'] = location.latitude, location.longitude
        except:
            print(address)

get_lat_longs(set_of_locs)
set_of_locs

GeocoderQuotaExceeded: The given key has gone over the requests limit in the 24 hour period or has submitted too many requests in too short a period of time.

In [15]:
df['locations'] = df['locations'].apply(get_lat_longs)